In [92]:
# Импортируем библиотеки
import pandas as pd
import warnings
import numpy as np

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer

from feature_engine.selection import DropCorrelatedFeatures, DropDuplicateFeatures

from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from imblearn import FunctionSampler
from sklearn.neighbors import LocalOutlierFactor

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn import svm

from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline

from sklearn.model_selection import RandomizedSearchCV

warnings.filterwarnings('ignore')
np.random.seed(seed=42)

In [93]:
test_csv = pd.read_csv('./datasets/test.csv')
train_csv = pd.read_csv('./datasets/train.csv')

display(test_csv.head())
display(train_csv.head())

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,8026_01,Mars,True,F/1534/S,TRAPPIST-1e,29.0,False,0.0,0.0,0.0,0.0,0.0,Losp Ruité
1,6734_03,Europa,True,C/249/S,55 Cancri e,37.0,True,0.0,0.0,0.0,0.0,0.0,Alhaino Sparsaxle
2,8186_04,Earth,True,G/1319/S,PSO J318.5-22,23.0,False,0.0,0.0,0.0,0.0,0.0,Rayley Jenkinnedy
3,4942_02,Earth,True,G/806/S,TRAPPIST-1e,36.0,False,0.0,0.0,0.0,0.0,0.0,Coreee Reenon
4,8640_01,Earth,False,F/1666/S,NaN,50.0,False,133.0,0.0,525.0,0.0,31.0,Diandy Hurchrisong


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,6234_01,Mars,False,F/1287/P,PSO J318.5-22,40.0,True,301.0,0.0,1310.0,4.0,NaN,Sters Bun,True
1,8826_01,Europa,False,C/290/P,55 Cancri e,13.0,False,1212.0,4485.0,0.0,4016.0,57.0,Marin Hednigic,False
2,4453_01,Europa,False,D/149/P,TRAPPIST-1e,28.0,False,0.0,2560.0,0.0,134.0,307.0,Azhasim Inguel,False
3,0602_03,Mars,True,F/125/P,TRAPPIST-1e,21.0,False,0.0,0.0,0.0,0.0,0.0,Rana Brine,True
4,8796_01,Earth,False,G/1418/S,TRAPPIST-1e,6.0,False,0.0,0.0,0.0,0.0,0.0,Daisya Traverdy,True


In [94]:
display(test_csv.info())
display(train_csv.info())
display(train_csv.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1739 entries, 0 to 1738
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   1739 non-null   object 
 1   HomePlanet    1701 non-null   object 
 2   CryoSleep     1692 non-null   object 
 3   Cabin         1696 non-null   object 
 4   Destination   1698 non-null   object 
 5   Age           1703 non-null   float64
 6   VIP           1692 non-null   object 
 7   RoomService   1700 non-null   float64
 8   FoodCourt     1700 non-null   float64
 9   ShoppingMall  1693 non-null   float64
 10  Spa           1701 non-null   float64
 11  VRDeck        1698 non-null   float64
 12  Name          1699 non-null   object 
dtypes: float64(6), object(7)
memory usage: 176.7+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6954 entries, 0 to 6953
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   6954 non-null   object 
 1   HomePlanet    6791 non-null   object 
 2   CryoSleep     6784 non-null   object 
 3   Cabin         6798 non-null   object 
 4   Destination   6813 non-null   object 
 5   Age           6811 non-null   float64
 6   VIP           6798 non-null   object 
 7   RoomService   6812 non-null   float64
 8   FoodCourt     6810 non-null   float64
 9   ShoppingMall  6792 non-null   float64
 10  Spa           6809 non-null   float64
 11  VRDeck        6807 non-null   float64
 12  Name          6794 non-null   object 
 13  Transported   6954 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 713.2+ KB


None

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,6811.000000,6812.000000,6810.000000,6792.000000,6809.000000,6807.000000
mean,28.758773,219.851879,458.781791,173.162691,302.198120,305.534156
std,14.370668,649.921373,1601.377860,598.108352,1120.863112,1148.581726
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,42.000000,70.750000,24.000000,56.000000,45.000000
max,79.000000,9920.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [95]:
# Создадим функцию для разделения колонок, содержащие сложные значения, на несколько колонок с простыми значениями
def split_complex_columns(df):
    modified_df = df

    modified_df = modified_df.join(modified_df['Cabin'].str.split('/', expand=True))
    modified_df = modified_df.drop(columns = ['Cabin'], axis = 1)
    modified_df = modified_df.rename(columns={0:'Deck', 1:'Num', 2:'Side'})

    modified_df = modified_df.join(modified_df['PassengerId'].str.split('_', expand=True))
    modified_df = modified_df.drop(columns = ['PassengerId'], axis = 1)
    modified_df = modified_df.rename(columns={0:'Group', 1:'GroupNumber'})
    modified_df = modified_df.drop(columns = ['GroupNumber'], axis = 1)

    modified_df = modified_df.join(modified_df['Name'].str.split(' ', expand=True))
    modified_df = modified_df.drop(columns = ['Name'], axis = 1)
    modified_df = modified_df.rename(columns={0:'Name', 1:'Surname'})
    modified_df = modified_df.drop(columns = ['Name'], axis = 1)

    return modified_df

In [96]:
# Применим функцию к train и test выборкам
train_splitted_columns = split_complex_columns(train_csv)
test_splitted_columns = split_complex_columns(test_csv)

display(train_splitted_columns)
display(test_splitted_columns)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,Group,Surname
0,Mars,False,PSO J318.5-22,40.0,True,301.0,0.0,1310.0,4.0,NaN,True,F,1287,P,6234,Bun
1,Europa,False,55 Cancri e,13.0,False,1212.0,4485.0,0.0,4016.0,57.0,False,C,290,P,8826,Hednigic
2,Europa,False,TRAPPIST-1e,28.0,False,0.0,2560.0,0.0,134.0,307.0,False,D,149,P,4453,Inguel
3,Mars,True,TRAPPIST-1e,21.0,False,0.0,0.0,0.0,0.0,0.0,True,F,125,P,0602,Brine
4,Earth,False,TRAPPIST-1e,6.0,False,0.0,0.0,0.0,0.0,0.0,True,G,1418,S,8796,Traverdy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6949,Mars,False,TRAPPIST-1e,24.0,False,2997.0,0.0,3.0,0.0,7.0,True,E,70,P,1182,Pota
6950,Europa,True,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,0.0,True,B,106,P,3396,Mirinanty
6951,Europa,False,55 Cancri e,19.0,False,0.0,1291.0,0.0,16139.0,687.0,False,B,48,S,1161,Anderking
6952,Europa,False,TRAPPIST-1e,32.0,False,0.0,89.0,0.0,8330.0,1.0,False,C,255,S,6921,Stersetery


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side,Group,Surname
0,Mars,True,TRAPPIST-1e,29.0,False,0.0,0.0,0.0,0.0,0.0,F,1534,S,8026,Ruité
1,Europa,True,55 Cancri e,37.0,True,0.0,0.0,0.0,0.0,0.0,C,249,S,6734,Sparsaxle
2,Earth,True,PSO J318.5-22,23.0,False,0.0,0.0,0.0,0.0,0.0,G,1319,S,8186,Jenkinnedy
3,Earth,True,TRAPPIST-1e,36.0,False,0.0,0.0,0.0,0.0,0.0,G,806,S,4942,Reenon
4,Earth,False,NaN,50.0,False,133.0,0.0,525.0,0.0,31.0,F,1666,S,8640,Hurchrisong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,Earth,False,TRAPPIST-1e,26.0,False,0.0,786.0,0.0,11.0,83.0,F,1160,P,5589,Hermanapp
1735,Earth,True,TRAPPIST-1e,15.0,False,0.0,0.0,0.0,0.0,0.0,G,1336,P,8229,Patthews
1736,Mars,False,TRAPPIST-1e,19.0,False,0.0,0.0,0.0,0.0,0.0,E,134,P,2234,Flité
1737,Earth,False,TRAPPIST-1e,29.0,False,272.0,615.0,0.0,0.0,1.0,G,42,P,0253,Cardner


In [97]:
# Посмотрим на количество пропусков в колонках train выборки
train_splitted_columns.isna().sum()

HomePlanet      163
CryoSleep       170
Destination     141
Age             143
VIP             156
RoomService     142
FoodCourt       144
ShoppingMall    162
Spa             145
VRDeck          147
Transported       0
Deck            156
Num             156
Side            156
Group             0
Surname         160
dtype: int64

In [98]:
# Выявили закономерность, что пассажиры с VIP обслуживанием, скорее всего, не располагались на палубе G
train_splitted_columns[train_splitted_columns['Deck'] == 'G']['VIP'].unique()

array([False, nan], dtype=object)

In [99]:
# Заполняем все пропуски в стобце VIP для палубы G значением False
train_splitted_columns.loc[(train_splitted_columns['Deck'] == 'G') & train_splitted_columns['VIP'].isna(), 'VIP'] = False
train_splitted_columns[train_splitted_columns['Deck'] == 'G']['VIP'].unique()

array([False], dtype=object)

In [100]:
# Выявили закономерность, что у пассажиров состоящих в одной и той же группе совпадает место отправки
train_groupped_by_group = train_splitted_columns.groupby('Group')
train_groupped_by_group.filter(lambda x: x['HomePlanet'].nunique() > 1)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,Group,Surname


In [101]:
# Места назначения для одной и той же группы могут быть разные
train_groupped_by_group.filter(lambda x: x['Destination'].nunique() > 1)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side,Group,Surname
1,Europa,False,55 Cancri e,13.0,False,1212.0,4485.0,0.0,4016.0,57.0,False,C,290,P,8826,Hednigic
4,Earth,False,TRAPPIST-1e,6.0,False,0.0,0.0,0.0,0.0,0.0,True,G,1418,S,8796,Traverdy
6,Earth,False,55 Cancri e,27.0,False,3.0,13.0,0.0,0.0,788.0,False,F,1683,P,8129,Casonston
11,NaN,False,TRAPPIST-1e,41.0,False,6.0,1.0,11.0,1046.0,0.0,False,F,736,S,3911,Petermandez
12,Earth,False,TRAPPIST-1e,3.0,False,0.0,0.0,0.0,0.0,0.0,True,G,857,P,5303,Lambleyoun
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,Europa,False,55 Cancri e,25.0,False,484.0,11441.0,0.0,4.0,5477.0,False,C,195,P,6340,Pressive
6946,Europa,False,TRAPPIST-1e,43.0,False,2.0,5329.0,0.0,7.0,0.0,True,B,245,S,6285,Whealeed
6947,Europa,True,TRAPPIST-1e,19.0,False,0.0,0.0,0.0,0.0,0.0,True,C,225,P,6971,Distured
6949,Mars,False,TRAPPIST-1e,24.0,False,2997.0,0.0,3.0,0.0,7.0,True,E,70,P,1182,Pota


In [102]:
# Пассажиры, с одной и той же фамилией, путешествующие в одной группе скорее всего являются членами семьи.
# Исходя из этого можно заполнить пропуски в соотвествующих колонках для семей
def fillna_in_group(x):
    index = x.first_valid_index()
    if (index is None):
        return x
    else:
        value = x[index]
        return x.fillna(value)

# Группировка по фамилии и группе
train_groupped_by_surname_and_group = train_splitted_columns.groupby(['Surname', 'Group'])
train_splitted_columns['HomePlanet'].update(train_groupped_by_surname_and_group['HomePlanet'].apply(lambda x: fillna_in_group(x)))
train_splitted_columns['Destination'].update(train_groupped_by_surname_and_group['Destination'].apply(lambda x: fillna_in_group(x)))
train_splitted_columns['CryoSleep'].update(train_groupped_by_surname_and_group['CryoSleep'].apply(lambda x: fillna_in_group(x)))
train_splitted_columns['VIP'].update(train_groupped_by_surname_and_group['VIP'].apply(lambda x: fillna_in_group(x)))
train_splitted_columns['Deck'].update(train_groupped_by_surname_and_group['Deck'].apply(lambda x: fillna_in_group(x)))

# Заполним пропуски в местах отправки для пассажиров из одной и той же группы
train_splitted_columns['HomePlanet'].update(train_groupped_by_group['HomePlanet'].apply(lambda x: fillna_in_group(x)))

# Пассажиры, находящиеся в состоянии криосна, врядли пользуются дополнительными сервисами,
# поэтому заполним пропуски для них нулями
train_splitted_columns.loc[train_splitted_columns['CryoSleep'] == True, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = \
    train_splitted_columns.loc[train_splitted_columns['CryoSleep'] == True, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(0)

In [103]:
# Посмотрим на количество оставшихся пропусков
display(train_splitted_columns.isna().sum())
print('Количество строк с пропусками', train_splitted_columns.isnull().any(axis=1).sum())

HomePlanet      103
CryoSleep       109
Destination      93
Age             143
VIP              71
RoomService      88
FoodCourt        90
ShoppingMall     86
Spa              91
VRDeck          104
Transported       0
Deck            100
Num             156
Side            156
Group             0
Surname         160
dtype: int64

Количество строк с пропусками 1193


In [104]:
# Выделим целевой признак из train выборки и удалим колонку 'Surname'
train_features = train_splitted_columns.drop(columns=['Transported', 'Surname'])
train_target = train_splitted_columns.Transported

display(train_features)
display(train_target)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side,Group
0,Mars,False,PSO J318.5-22,40.0,True,301.0,0.0,1310.0,4.0,NaN,F,1287,P,6234
1,Europa,False,55 Cancri e,13.0,False,1212.0,4485.0,0.0,4016.0,57.0,C,290,P,8826
2,Europa,False,TRAPPIST-1e,28.0,False,0.0,2560.0,0.0,134.0,307.0,D,149,P,4453
3,Mars,True,TRAPPIST-1e,21.0,False,0.0,0.0,0.0,0.0,0.0,F,125,P,0602
4,Earth,False,TRAPPIST-1e,6.0,False,0.0,0.0,0.0,0.0,0.0,G,1418,S,8796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6949,Mars,False,TRAPPIST-1e,24.0,False,2997.0,0.0,3.0,0.0,7.0,E,70,P,1182
6950,Europa,True,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,0.0,B,106,P,3396
6951,Europa,False,55 Cancri e,19.0,False,0.0,1291.0,0.0,16139.0,687.0,B,48,S,1161
6952,Europa,False,TRAPPIST-1e,32.0,False,0.0,89.0,0.0,8330.0,1.0,C,255,S,6921


0        True
1       False
2       False
3        True
4        True
        ...  
6949     True
6950     True
6951    False
6952    False
6953     True
Name: Transported, Length: 6954, dtype: bool

In [105]:
# Оценим сбалансированность train выборки
train_target.value_counts()

True     3502
False    3452
Name: Transported, dtype: int64

In [106]:
# Выделим категориальные, бинарные и числовые колонки
category_columns = ['HomePlanet', 'Destination', 'Deck']
binary_columns = ['CryoSleep', 'VIP', 'Side']
numeric_column = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Num', 'Group']

In [107]:
# Для категориальных колонок будем использовать OrdinalEncoder, а для бинарных OneHotEncoder
numeric_columns_pipeline = Pipeline([('scaling', StandardScaler()),
                                     ('drop_correlations', DropCorrelatedFeatures())])

columns_pipeline = ColumnTransformer([('handle_numeric_columns', numeric_columns_pipeline, numeric_column),
                                      ('category_columns_encoding', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), category_columns),
                                      ('binary_columns_encoding', OneHotEncoder(handle_unknown='ignore', drop='first'), binary_columns)],
                                      remainder='passthrough')

columns_pipeline

ColumnTransformer(remainder='passthrough',
                  transformers=[('handle_numeric_columns',
                                 Pipeline(steps=[('scaling', StandardScaler()),
                                                 ('drop_correlations',
                                                  DropCorrelatedFeatures())]),
                                 ['Age', 'RoomService', 'FoodCourt',
                                  'ShoppingMall', 'Spa', 'VRDeck', 'Num',
                                  'Group']),
                                ('category_columns_encoding',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['HomePlanet', 'Destination', 'Deck']),
                                ('binary_columns_encoding',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore'),
                                 ['CryoSleep', 'VIP', 'Side'])])

In [108]:
# Добавим обработку пропусков и дубликатов фич
pipeline = make_pipeline(columns_pipeline, DropDuplicateFeatures(), KNNImputer())
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('handle_numeric_columns',
                                                  Pipeline(steps=[('scaling',
                                                                   StandardScaler()),
                                                                  ('drop_correlations',
                                                                   DropCorrelatedFeatures())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck', 'Num',
                                                   'Group']),
                                                 ('category_columns_encoding',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['HomePlanet', 'Destination',
                                                   'Deck']),
                                                 ('binary_columns_encoding',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['CryoSleep', 'VIP',
                                                   'Side'])])),
                ('dropduplicatefeatures', DropDuplicateFeatures()),
                ('knnimputer', KNNImputer())])

In [109]:
# Добавим обработку выбросов и шаг для модели классификатора
def handle_outliers(features, target):
    outlier_index =  LocalOutlierFactor().fit_predict(features)
    return features[outlier_index == 1], target[outlier_index == 1]

pipeline.steps.append(('outlier_detector', FunctionSampler(func=handle_outliers)))
pipeline.steps.append(('classification', DummyClassifier()))
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('handle_numeric_columns',
                                                  Pipeline(steps=[('scaling',
                                                                   StandardScaler()),
                                                                  ('drop_correlations',
                                                                   DropCorrelatedFeatures())]),
                                                  ['Age', 'RoomService',
                                                   'FoodCourt', 'ShoppingMall',
                                                   'Spa', 'VRDeck', 'Num',
                                                   'Group']),
                                                 ('category_columns_encoding',
                                                  OrdinalEncoder(handle_unknown...
                                                  ['HomePlanet', 'Destination',
                                                   'Deck']),
                                                 ('binary_columns_encoding',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['CryoSleep', 'VIP',
                                                   'Side'])])),
                ('dropduplicatefeatures', DropDuplicateFeatures()),
                ('knnimputer', KNNImputer()),
                ('outlier_detector',
                 FunctionSampler(func=<function handle_outliers at 0x00000207CBCB49D0>)),
                ('classification', DummyClassifier())])

In [110]:
# Посмотри на шаги получившегося пайплайна
pipeline.steps

[('columntransformer',
  ColumnTransformer(remainder='passthrough',
                    transformers=[('handle_numeric_columns',
                                   Pipeline(steps=[('scaling', StandardScaler()),
                                                   ('drop_correlations',
                                                    DropCorrelatedFeatures())]),
                                   ['Age', 'RoomService', 'FoodCourt',
                                    'ShoppingMall', 'Spa', 'VRDeck', 'Num',
                                    'Group']),
                                  ('category_columns_encoding',
                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                  unknown_value=-1),
                                   ['HomePlanet', 'Destination', 'Deck']),
                                  ('binary_columns_encoding',
                                   OneHotEncoder(drop='first',
                      

In [111]:
# Опишем и зададим параметры для RandomizedSearchCV
def create_randomized_search_cv(pipeline, iterations_count, parameters=None):
    if parameters is None:
        params = [
            {
                'classification': [RandomForestClassifier()],
                'classification__n_estimators': [2,10,100,200,300],
                'classification__max_depth': [3,5,15,25,50],
                'columntransformer__handle_numeric_columns__scaling': [None],
                'knnimputer': [KNNImputer(), IterativeImputer()]
            },
            {
                'classification': [CatBoostClassifier(verbose=1)],
                'classification__n_estimators': [2,10,100,200,300],
                'classification__max_depth': [3,5,15,16],
                'columntransformer__handle_numeric_columns__scaling': [None],
                'knnimputer': [KNNImputer(), IterativeImputer()]
            },
            {
                'classification': [svm.SVC()],
                'classification__kernel': ['sigmoid', 'rbf', 'poly'],
                'classification__gamma': ['auto', 'scale'],
                'classification__degree': [3,6],
                'columntransformer__handle_numeric_columns__scaling': [MinMaxScaler(), StandardScaler()],
                'knnimputer': [KNNImputer(), IterativeImputer()]
            },
            {
                'classification': [LGBMClassifier(n_jobs=-1)],
                'classification__n_estimators': [100,200,300],
                'classification__num_leaves': [31,50,100,200],
                'columntransformer__handle_numeric_columns__scaling': [None],
                'knnimputer': [KNNImputer(), IterativeImputer()]
            }
        ]
    else:
        params = parameters

    grid = RandomizedSearchCV(pipeline,
                              params,
                              n_iter = iterations_count,
                              cv = 5,
                              verbose = 1,
                              scoring = 'roc_auc',
                              n_jobs=-1)

    return grid


In [112]:
grid = create_randomized_search_cv(pipeline, 30)

In [113]:
%%time

grid.fit(train_features, train_target)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Learning rate set to 0.09953
0:	learn: 0.6489619	total: 269ms	remaining: 53.6s
1:	learn: 0.6123374	total: 279ms	remaining: 27.7s
2:	learn: 0.5823363	total: 289ms	remaining: 19s
3:	learn: 0.5595224	total: 296ms	remaining: 14.5s
4:	learn: 0.5447406	total: 306ms	remaining: 11.9s
5:	learn: 0.5287047	total: 315ms	remaining: 10.2s
6:	learn: 0.5150441	total: 322ms	remaining: 8.87s
7:	learn: 0.5037925	total: 329ms	remaining: 7.89s
8:	learn: 0.4936685	total: 336ms	remaining: 7.13s
9:	learn: 0.4853044	total: 343ms	remaining: 6.52s
10:	learn: 0.4810834	total: 350ms	remaining: 6.01s
11:	learn: 0.4750914	total: 360ms	remaining: 5.64s
12:	learn: 0.4691350	total: 368ms	remaining: 5.29s
13:	learn: 0.4635202	total: 375ms	remaining: 4.98s
14:	learn: 0.4586972	total: 382ms	remaining: 4.71s
15:	learn: 0.4530156	total: 389ms	remaining: 4.47s
16:	learn: 0.4505324	total: 396ms	remaining: 4.26s
17:	learn: 0.4470521	total: 403ms	remaining: 4.08s
18:

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('handle_numeric_columns',
                                                                               Pipeline(steps=[('scaling',
                                                                                                StandardScaler()),
                                                                                               ('drop_correlations',
                                                                                                DropCorrelatedFeatures())]),
                                                                               ['Age',
                                                                                'RoomService',
                                                                                'FoodCourt',
                                                                                'ShoppingMall',
                                                                                'Spa',
                                                                                'VRDeck',
                                                                                'Num',
                                                                                'Group']),
                                                                              ('category_columns_encod...
                                         'columntransformer__handle_numeric_columns__scaling': [MinMaxScaler(),
                                                                                                StandardScaler()],
                                         'knnimputer': [KNNImputer(),
                                                        IterativeImputer()]},
                                        {'classification': [LGBMClassifier()],
                                         'classification__n_estimators': [100,
                                                                          200,
                                                                          300],
                                         'classification__num_leaves': [31, 50,
                                                                        100,
                                                                        200],
                                         'columntransformer__handle_numeric_columns__scaling': [None],
                                         'knnimputer': [KNNImputer(),
                                                        IterativeImputer()]}],
                   scoring='roc_auc', verbose=1)

In [114]:
# Посмотрим лучшие параметры
grid.best_params_

{'knnimputer': KNNImputer(),
 'columntransformer__handle_numeric_columns__scaling': None,
 'classification__n_estimators': 200,
 'classification__max_depth': 5,
 'classification': <catboost.core.CatBoostClassifier at 0x207c5815210>}

In [115]:
# Посмотрим получившуюся лучшую метрику
grid.best_score_

0.8948793920733398

In [117]:
# Посчитаем предсказания на тестовой выборке
predictions = grid.predict(test_splitted_columns)
predictions

array(['True', 'True', 'True', ..., 'False', 'False', 'False'],
      dtype=object)

In [118]:
    # Сформируем результирующий датафрейм и сохраним его в файл на диск для предоставления результатов
result = pd.concat([test_csv['PassengerId'], pd.Series(predictions, name='Transported')], axis=1)
display(result)
result.to_csv('./datasets/result.csv', index=False)


,PassengerId,Transported
0,8026_01,True
1,6734_03,True
2,8186_04,True
3,4942_02,True
4,8640_01,False
...,...,...
1734,5589_01,False
1735,8229_01,True
1736,2234_03,False
1737,0253_01,False
